In [68]:
#recovering dumped data from Hello ML
import joblib

forest_reg = joblib.load("my_model.pkl")
housing_prepared = joblib.load("housing_prepared.pkl")
housing_labels = joblib.load("housing_labels.pkl")
num_attribs = joblib.load("num_attribs.pkl")

from sklearn.ensemble import RandomForestRegressor
import numpy as np

In [2]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]}, #evaluate 3x4 hyperparameters combinations
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}, #3x2 hyperparameters combo w/o bootstrap
  ]#18 combos of RandomForestRegressor hyperparam values

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5, #five folds crossVal -> 18*5 rounds of training
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

grid_search.fit(housing_prepared, housing_labels)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [4]:
grid_search.best_params_

{'max_features': 6, 'n_estimators': 30}

In [ ]:
# n_estimators reached maximum, so it would probably improve again if we gridsearch with higher values

In [6]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [10, 30, 40], 'max_features': [2, 4, 6, 8]}, #evaluate 3x4 hyperparameters combinations
    {'bootstrap': [False], 'n_estimators': [10, 3], 'max_features': [2, 3, 4]}, #3x2 hyperparameters combo w/o bootstrap
  ]#18 combos of RandomForestRegressor hyperparam values

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5, #five folds crossVal -> 18*5 rounds of training
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

grid_search.fit(housing_prepared, housing_labels)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [10, 30, 40]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [10, 3]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [12]:
grid_search.best_params_

{'max_features': 6, 'n_estimators': 40}

In [16]:
grid_search.best_estimator_
#wont give the same output as the book

RandomForestRegressor(max_features=6, n_estimators=40)

In [21]:
# we could continue

#eval scores:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"],cvres["params"]):
    print(np.sqrt(-mean_score),params)

55864.24054531863 {'max_features': 2, 'n_estimators': 10}
52724.08862198114 {'max_features': 2, 'n_estimators': 30}
52532.2687970472 {'max_features': 2, 'n_estimators': 40}
52750.01999868154 {'max_features': 4, 'n_estimators': 10}
50085.50879013046 {'max_features': 4, 'n_estimators': 30}
50030.46590548733 {'max_features': 4, 'n_estimators': 40}
51770.57434486717 {'max_features': 6, 'n_estimators': 10}
49848.08333613207 {'max_features': 6, 'n_estimators': 30}
49591.84767619493 {'max_features': 6, 'n_estimators': 40}
51942.1206218345 {'max_features': 8, 'n_estimators': 10}
50110.945029441325 {'max_features': 8, 'n_estimators': 30}
50110.520869305 {'max_features': 8, 'n_estimators': 40}
54028.1041201624 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
63450.13578054074 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
52697.59529422097 {'bootstrap': False, 'max_features': 3, 'n_estimators': 10}
60509.50661435576 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3

In [47]:
#finding best score in a very spaghetti way
bestScore = None
for mean_score, params in zip(cvres["mean_test_score"],cvres["params"]):
    
    if bestScore == None:
        bestScore=bestScore=np.sqrt(-mean_score)
        bestScoreParams = params
    if np.sqrt(-mean_score)<bestScore:
        bestScore=np.sqrt(-mean_score)
        bestScoreParams = params
        
bestScore,bestScoreParams

(49591.84767619493, {'max_features': 6, 'n_estimators': 40})

In [48]:
#The best score found is slightly better than the mean fold RMSE. Yay!

In [51]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

array([7.24962136e-02, 6.76292673e-02, 4.18761392e-02, 1.73247055e-02,
       1.60470493e-02, 1.76013630e-02, 1.67832165e-02, 3.35789596e-01,
       6.17505380e-02, 1.07876839e-01, 7.27979922e-02, 6.40128569e-03,
       1.57987391e-01, 1.17625974e-04, 2.78924405e-03, 4.73153367e-03])

In [71]:
extra_attribs = ["rooms_per_household","population_per_household","bedrooms_per_room"]
cat_encoder = joblib.load("cat_encoder.pkl")
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances,attributes), reverse=True)
#shouldn't have separated into two notebooks right?

[(0.3357895958388674, 'median_income'),
 (0.1579873907602274, 'INLAND'),
 (0.10787683933014783, 'population_per_household'),
 (0.07279799221003846, 'bedrooms_per_room'),
 (0.07249621356507693, 'longitude'),
 (0.06762926730517166, 'latitude'),
 (0.0617505380352779, 'rooms_per_household'),
 (0.04187613921350518, 'housing_median_age'),
 (0.0176013629798095, 'population'),
 (0.01732470553744376, 'total_rooms'),
 (0.016783216514580833, 'households'),
 (0.01604704932074911, 'total_bedrooms'),
 (0.006401285690025608, '<1H OCEAN'),
 (0.0047315336730947505, 'NEAR OCEAN'),
 (0.0027892440521965923, 'NEAR BAY'),
 (0.00011762597378716642, 'ISLAND')]

In [72]:
#At this point you can improve the model by removing less relevant attributes
#for example, ocean proximity can be dropped and replaced with a 'INLAND' flag?

In [73]:
final_model = grid_search.best_estimator_
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)

final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)   # => evaluates to 47,730.2

NameError: name 'strat_test_set' is not defined